![Vidi_Waterflux_Banner](https://raw.githubusercontent.com/ITC-Water-Resources/Vidi-waterflux-merch/refs/heads/main/jupyter/Vidi_Waterflux_Banner.png)
*Roelof Rietbroek, Sedigheh Karimi, Amin Shakya EGU 2025*

# Compute spectral basin averages of GRACE derived TWS over the specified basins

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import geopandas as gpd
import shxarray
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from geoslurp import GeoslurpManager

#local python tools
from common.config import read_config

In [20]:
shxengine='shtns' #requires shtns backend but is much quicker
nmax=120
conf=read_config()
datadir=conf['dataroot']


In [3]:
#load basin coefficients (truncated)
ncbasins=os.path.join(datadir,conf['ncbasin'])
dsbasins_sh=xr.open_dataset(ncbasins).sh.truncate(nmax=nmax)
display(dsbasins_sh)

<xarray.Dataset> Size: 15MB
Dimensions:  (nm: 14641, name: 124)
Coordinates:
    n        (nm) int64 117kB 0 1 1 1 2 2 2 2 ... 120 120 120 120 120 120 120
    m        (nm) int64 117kB ...
  * name     (name) <U39 19kB 'SETIT' ... 'TIBETAN PLATEAU'
Dimensions without coordinates: nm
Data variables:
    basins   (name, nm) float64 15MB ...

In [9]:
#load static gravity field from database

gscon=GeoslurpManager(dbalias='tunnelmarge',readonly_user=False)

In [21]:
icgemstatic=gscon.execute("select uri from shxarray.icgemstatic WHERE data->>'name' = 'GOCO06s' ").first()
staticfile=icgemstatic.uri.replace('${LOCALDATAROOT}',gscon.clientconf.dataroot)
dsstatic=xr.open_dataset(staticfile,engine='icgem').sh.truncate(nmax)


/home/roelof/cld_UTwente/Soft/shxarray-git/src/shxarray/io/icgem.py:148: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'nm' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  ds=xr.Dataset(data_vars=dict(cnm=(shp,cnm,get_cfatts("stokes")),sigcnm=(shp,sigcnm,get_cfatts("stokes stdv"))),coords=coords,attrs=attr)


In [22]:
dsstatic

<xarray.Dataset> Size: 586kB
Dimensions:  (nm: 14641)
Coordinates:
  * nm       (nm) object 117kB MultiIndex
  * n        (nm) int64 117kB 0 1 1 1 2 2 2 2 ... 120 120 120 120 120 120 120
  * m        (nm) int64 117kB 0 0 1 -1 0 1 -1 ... 118 -118 119 -119 120 -120
Data variables:
    cnm      (nm) float64 117kB ...
    sigcnm   (nm) float64 117kB ...
Attributes:
    nmaxfile:   300
    nmax:       300
    format:     icgem
    norm:       fully_normalized
    gm:         398600441500000.0
    re:         6378136.3
    modelname:  GOCO06s